In [ ]:
import gc 
gc.collect()

# Temporal Response Function (TRF) - Visualization

### Modules 

In [ ]:
import mne 
import pickle
import os
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
%matplotlib inline
import seaborn as sns
from scipy import stats
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf

### Define dictionaries, subject, & conditions

In [ ]:
# set path to main folder
%cd ~/
%pwd

# path to data files
data_path = "/Volumes/Elements/data_mne/"

# subject 
subj = ['Bou_Ni'] 
sub_idx = 0 # subject number

# frequency band 
band_name = "Low Frequencies"
band = "deltatheta" 

# list of conditions
condition_list = ['produce_music', 'perceive_music_produced', 'produce_speech', 'perceive_speech_produced']
contrasts = [["produce_music", "perceive_music_produced"], ["produce_speech", "perceive_speech_produced"]]
pairs = [["music","produce_music"], ["music", "perceive_music_produced"], ["speech", "produce_speech"], ["speech", "perceive_speech_produced"]]
contrasts = [["produce_music", "perceive_music_produced"], ["produce_speech", "perceive_speech_produced"]]
condition_names = ["Music \n Production", "Music \n Perception", "Speech \n Production", "Speech \n Perception"]
contrast_names = ["Music Production And Music Perception", "Speech Production and Speech Perception"]

condition_list_control = ['perceive_music_new', 'perceive_music_newrepetition', 'perceive_speech_new', 'perceive_speech_newrepetition']
contrasts_control = [["perceive_music_new", "perceive_music_newrepetition"], ["perceive_speech_new", "perceive_speech_newrepetition"]]
pairs_control = [["perceive_music_new", "perceive_music_newrepetition"], ["speech_new", "perceive_speech_new"], ["speech_new", "perceive_speech_newrepetition"]]
condition_names_control = ["Music \n Perception \n Control", "Music \n Perception \n Control Repetition", "Speech \n Perception \n Control", "Speech \n Perception \n Control Repetition", ]
contrast_names_control = ["Music First Hearing And Music Second Hearing", "Speech First Hearing and Speech Second Hearing"]

In [ ]:
# recording path 
recording_path = data_path + subj[sub_idx] + "/wavfiles/"
print(recording_path)

# matlabfiles path 
features_path = data_path + subj[sub_idx] + "/stimulusfeatures/"
print(features_path)

# dictionary path 
dictionary_path = data_path + subj[sub_idx] + "/dictionary"

# plot path 
plot_path = data_path + subj[sub_idx] + "/plots/Temporal-Response-Function-Plots/"
print(plot_path)

# freesurfer path 
fs_path = data_path + subj[sub_idx] + '/freesurfer/Bou_Ni/elec_recon/'

### Load raw data and channel information 

In [ ]:
# load in raw 
raw = {}

for condition in condition_list:
    preprocessed_path = data_path + subj[sub_idx] + "/preprocessed/" + condition + "/"
    
    for files in os.listdir(preprocessed_path):
        filename = "day1_bipolar_raw_preprocessed.fif"
        if filename in files:
            path = preprocessed_path + files + '/'
            raw[condition] =  mne.io.read_raw_fif(path, preload=True)

In [ ]:
# channel names (picks)
picks = raw['produce_music'].ch_names

# define channels in the auditory cortex 
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
print('channels in auditory cortex: ', picks_H)

# assign channels to LH and RH primary and non-primary auditory cortex 
picks_primary_RH = ['H5-H6', 'H6-H7', 'H7-H8', 'H8-H9', 'H9-H10', 'H10-H11']
picks_primary_LH = [ "H'5-H'6", "H'6-H'7", "H'7-H'8", "H'8-H'9"]
picks_nonprimary_RH = ['H11-H12', 'H12-H13']
picks_nonprimary_LH = [ "H'9-H'10", "H'10-H'11", "H'11-H'12"]

picks_subset = [picks_primary_LH, picks_nonprimary_LH, picks_primary_RH, picks_nonprimary_RH,]
picks_subset_names = ["Left Primary Auditory Cortex", "Left Associative Auditory Cortex", "Right Primary Auditory Cortex", "Right Associative Auditory Cortex"] 


### Load accoustic regressors and TRF data

In [ ]:
# regressors  
with open(f"{dictionary_path}/TRF_0906/TRF_regressors_deltatheta_day1.pickle", 'rb') as f:
        regressors1 = pickle.load(f)

with open(f"{dictionary_path}/TRF_0906/TRF_regressors_deltatheta_day2.pickle", 'rb') as f:
        regressors2 = pickle.load(f)

In [ ]:
# load TRF crossvalidated data for H-channels 
with open(f"{dictionary_path}/TRF_0906/TRF_results_deltatheta_day1_crossvalidate.pickle", 'rb') as f:
        TRF1 = pickle.load(f)

with open(f"{dictionary_path}/TRF_0906/TRF_results_deltatheta_day2_crossvalidate.pickle", 'rb') as f:
        TRF2 = pickle.load(f)

In [ ]:
# parameters 
labels = regressors1['music'][1]
fs = 100
scoring = 'r2'
tw = [-0.2, 0.5]                                            #in sec

# Compare conditions

## Self Produced Speech and Music: Production - Perception 

In [ ]:
# concatenate data from two days
TRF = {}
for condition in condition_list: 
        TRF[condition] = {}
        print(condition)
        for pick in picks_H: 
                TRF[condition][pick] = {}
                TRF[condition][pick]['score'] = np.concatenate((TRF1[condition][pick]['score'], TRF2[condition][pick]['score']), axis=0)
                TRF[condition][pick]['coefs'] = np.concatenate((TRF1[condition][pick]['coefs'], TRF2[condition][pick]['coefs']), axis=0)
                TRF[condition][pick]['predicted_Y'] = np.concatenate((TRF1[condition][pick]['predicted_Y'], TRF2[condition][pick]['predicted_Y']), axis=0)
                TRF[condition][pick]['times'] = TRF1[condition][pick]['times'] # same across two days 


In [ ]:
# load TRF for main analysis for day1
TRF_day1 = {}
for condition in condition_list: 
        TRF_day1[condition] = {}
        print(condition)
        for pick in picks_H: 
                TRF_day1[condition][pick] = {}
                TRF_day1[condition][pick]['score'] = TRF1[condition][pick]['score']
                TRF_day1[condition][pick]['coefs'] = TRF1[condition][pick]['coefs']
                TRF_day1[condition][pick]['predicted_Y'] = TRF1[condition][pick]['predicted_Y']
                TRF_day1[condition][pick]['times'] = TRF1[condition][pick]['times'] 

# load TRF for main analysis for day1
TRF_day2 = {}
for condition in condition_list: 
        TRF_day2[condition] = {}
        print(condition)
        for pick in picks_H: 
                TRF_day2[condition][pick] = {}
                TRF_day2[condition][pick]['score'] = TRF2[condition][pick]['score']
                TRF_day2[condition][pick]['coefs'] = TRF2[condition][pick]['coefs']
                TRF_day2[condition][pick]['predicted_Y'] = TRF2[condition][pick]['predicted_Y']
                TRF_day2[condition][pick]['times'] = TRF2[condition][pick]['times'] 

In [ ]:
# TRF shape 
pick = 'H5-H6'
i_reg = 5 # loadness derivative
colors = ['purple', 'plum', '#008000', '#90EE90']

fig, ax = plt.subplots(figsize=(8,6))
fig.subplots_adjust(top=0.85)

for n, condition in enumerate(condition_list):
    times = TRF[condition][pick]['times']

    coefs_pick = TRF[condition][pick]["coefs"]
    coefs_mean = np.mean(coefs_pick[:, i_reg, :], axis=0)
    ax.plot(times, coefs_mean, label=condition_names[n], color=colors[n], linewidth=3)


ax.hlines(0, times[0], times[1], colors='gray', linestyles='--')
ax.vlines(0, np.min(values), np.max(values), color='black', linestyle='--', alpha=0.5)  # t = 0
ax.vlines(0.1, np.min(values), np.max(values), color='gray', linestyle='--', alpha=0.5)  # t = 100ms

ax.legend(loc='upper right', prop={'size': 8})
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude")
# plt.savefig(plot_path + f"TRF_efferencecopy_averagedchannels_{labels[i_reg]}_deltatheta_{day}_trf-plot.jpg", bbox_inches='tight')
plt.show()  

In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]
channel_assignment = [0, 6, 8, 12, len(picks_H)]

for n, contrast in enumerate(contrasts):
    print(contrast)
    if "music" in contrast[0]: 
        color_prod = 'purple'
        color_perc = 'plum'
    elif "speech" in contrast[0]: 
        color_prod = '#008000'
        color_perc = '#90EE90'

        
    data_production = np.array([TRF[contrast[0]][pick]['score'] for pick in picks_H])
    data_perception = np.array([TRF[contrast[1]][pick]['score'] for pick in picks_H])

    mean_production = np.mean(data_production, axis=1)
    se_production = np.std(data_production, axis=1, ddof=1) / np.sqrt(len(data_production[0]))

    mean_perception = np.mean(data_perception, axis=1)
    se_perception = np.std(data_perception, axis=1, ddof=1) / np.sqrt(len(data_perception[0]))

    # t-test
    t, p = stats.stats.ttest_rel(data_production.T, data_perception.T)
    p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)    
    print(p_corrected < 0.05)
    significance_threshold = 0.05
    significant_channels = np.where(p_corrected < significance_threshold)[0]

    # plot
    fig, ax = plt.subplots(figsize=(12, 8))
    ax2 = ax.twinx()  

    ## plot data points 
    jitter_width = 0.15 
    for i in range(len(data_production)):
        jitter_prod = np.random.uniform(-jitter_width, jitter_width, size=len(data_production[i]))
        jitter_perc = np.random.uniform(-jitter_width, jitter_width, size=len(data_perception[i]))

        ax.scatter(data_production[i], np.full(len(data_production[i]), i) + jitter_prod,
                alpha=0.3, color=color_prod, s=10)
        ax.scatter(data_perception[i], np.full(len(data_perception[i]), i) + jitter_perc,
                alpha=0.3, color=color_perc, s=10)
        
    ## plot means, se  
    for c in range(len(channel_assignment)-1):
        ax.errorbar(mean_production[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_production[channel_assignment[c]:channel_assignment[c+1]], marker='o', linestyle='-', color=color_prod, capsize=8)
        ax.errorbar(mean_perception[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_perception[channel_assignment[c]:channel_assignment[c+1]], marker='s', linestyle='-', color=color_perc, capsize=8)
    

    ## plot significant differences
    for s in range(len(picks_H)):
        if p_corrected[s] < 0.05:
            x_pos = -0.225
            y_pos = s - 0.15
            ax.text(x_pos, y_pos, "*", fontsize=20, color='black', ha='center', va='center', zorder=3)

    ax.set_xlabel('r$^2$ Coefficients', fontsize=14)
    ax.set_ylabel('Channels', fontsize=14)
    ax.set_xlim([-0.25, 0.55])
    ax.set_yticks(np.arange(len(picks_H)))
    ax.set_yticklabels(picks_H, fontsize=10)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    for channel_pos_index in [[0,5], [5,7], [7,11]]: 
        ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1)
    ax2.set_ylim(ax.get_ylim()) 
    channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
    ax2.set_yticks(channel_midpoints)  
    ax2.set_yticklabels(brain_label, size=10, va='center')  

    ax.legend(["Production", "Perception"])
    plt.show()

In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]
channel_assignment = [0, 6, 8, 12, len(picks_H)]

for n, contrast in enumerate(contrasts):
    print(contrast)
    if "music" in contrast[0]: 
        color_prod = 'purple'
        color_perc = 'plum'
    elif "speech" in contrast[0]: 
        color_prod = '#008000'
        color_perc = '#90EE90'

        
    data_production = np.array([TRF[contrast[0]][pick]['score'] for pick in picks_H])
    data_perception = np.array([TRF[contrast[1]][pick]['score'] for pick in picks_H])

    mean_production = np.mean(data_production, axis=1)
    se_production = np.std(data_production, axis=1, ddof=1) / np.sqrt(len(data_production[0]))

    mean_perception = np.mean(data_perception, axis=1)
    se_perception = np.std(data_perception, axis=1, ddof=1) / np.sqrt(len(data_perception[0]))

    # t-test
    t, p = stats.stats.ttest_rel(data_production.T, data_perception.T)
    p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)    
    print(p_corrected < 0.05)
    significance_threshold = 0.05
    significant_channels = np.where(p_corrected < significance_threshold)[0]

    # plot
    fig, ax = plt.subplots(figsize=(12, 8))
    ax2 = ax.twinx()  
  
    ## plot means, se  
    for c in range(len(channel_assignment)-1):
        ax.errorbar(mean_production[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_production[channel_assignment[c]:channel_assignment[c+1]], marker='o', color=color_prod, capsize=8)
        ax.errorbar(mean_perception[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_perception[channel_assignment[c]:channel_assignment[c+1]], marker='o', color=color_perc, capsize=8)
    
    ## plot significant differences
    for s in range(len(picks_H)):
        if p_corrected[s] < 0.05:
            x_pos = - 0.025
            y_pos = s - 0.15
            ax.text(x_pos, y_pos, "*", fontsize=20, color='black', ha='center', va='center', zorder=3)

    #ax.set_title(f'r$^2$ Coeffients for {contrast_names[n]} \n', fontsize=16)
    ax.set_xlabel('\n r$^2$-coefficients', fontsize=16)
    ax.tick_params(axis='x', labelsize=16)
    ax.set_ylabel('Channels', fontsize=16)
    ax.set_xlim([-0.05, 0.42])
    ax.set_yticks(np.arange(len(picks_H)))
    ax.set_yticklabels(picks_H, fontsize=14)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    for channel_pos_index in [[0,5], [5,7], [7,11]]: 
        ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1)
    ax2.set_ylim(ax.get_ylim()) 
    channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
    ax2.set_yticks(channel_midpoints)  
    ax2.set_yticklabels(brain_label, size=14, va='center')  

    legend_handles = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor=color_prod, markersize=15, label='Production'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor=color_perc, markersize=15, label='Perception')
    ]

    ax.legend(handles=legend_handles, fontsize=13) 
plt.show()

In [ ]:
# plot seperatly for the days 
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]
channel_assignment = [0, 6, 8, 12, len(picks_H)]

for day, TRF in zip(['Day1', 'Day2'], [TRF1, TRF2]):
    for n, contrast in enumerate(contrasts):
        print(contrast)
        if "music" in contrast[0]: 
            color_prod = 'purple'
            color_perc = 'plum'
        elif "speech" in contrast[0]: 
            color_prod = '#008000'
            color_perc = '#90EE90'

        data_production = np.array([TRF[contrast[0]][pick]['score'] for pick in picks_H])
        data_perception = np.array([TRF[contrast[1]][pick]['score'] for pick in picks_H])

        mean_production = np.mean(data_production, axis=1)
        se_production = np.std(data_production, axis=1, ddof=1) / np.sqrt(len(data_production[0]))

        mean_perception = np.mean(data_perception, axis=1)
        se_perception = np.std(data_perception, axis=1, ddof=1) / np.sqrt(len(data_perception[0]))

        # t-test
        t, p = stats.stats.ttest_rel(data_production.T, data_perception.T)
        p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)    
        print(p_corrected < 0.05)
        significance_threshold = 0.05
        significant_channels = np.where(p_corrected < significance_threshold)[0]

        # plot
        fig, ax = plt.subplots(figsize=(12, 8))
        ax2 = ax.twinx()  

        ## plot data points 
        jitter_width = 0.15 
        for i in range(len(data_production)):
            jitter_prod = np.random.uniform(-jitter_width, jitter_width, size=len(data_production[i]))
            jitter_perc = np.random.uniform(-jitter_width, jitter_width, size=len(data_perception[i]))

            ax.scatter(data_production[i], np.full(len(data_production[i]), i) + jitter_prod,
                    alpha=0.3, color=color_prod, s=10)
            ax.scatter(data_perception[i], np.full(len(data_perception[i]), i) + jitter_perc,
                    alpha=0.3, color=color_perc, s=10)
            
        ## plot means, se  
        for c in range(len(channel_assignment)-1):
            ax.errorbar(mean_production[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_production[channel_assignment[c]:channel_assignment[c+1]], marker='o', linestyle='-', color=color_prod, capsize=8)
            ax.errorbar(mean_perception[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_perception[channel_assignment[c]:channel_assignment[c+1]], marker='s', linestyle='-', color=color_perc, capsize=8)

        ## plot significant differences
        for s in range(len(picks_H)):
            if p_corrected[s] < 0.05:
                x_pos = -0.225
                y_pos = s - 0.15
                ax.text(x_pos, y_pos, "*", fontsize=20, color='black', ha='center', va='center', zorder=3)

        #ax.set_title(f'r$^2$ Coeffients for {contrast_names[n]} for {day} \n', fontsize=16)
        ax.set_xlabel('r$^2$ Coefficients', fontsize=14)
        ax.set_ylabel('Channels', fontsize=14)
        ax.set_xlim([-0.25, 0.55])
        ax.set_yticks(np.arange(len(picks_H)))
        ax.set_yticklabels(picks_H, fontsize=10)
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        for channel_pos_index in [[0,5], [5,7], [7,11]]: 
            ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1)
        ax2.set_ylim(ax.get_ylim()) 
        channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
        ax2.set_yticks(channel_midpoints)  
        ax2.set_yticklabels(brain_label, size=10, va='center')  

        ax.legend(["Production", "Perception"])
        plt.show()

In [ ]:
# plot seperatly for the days 
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]
channel_assignment = [0, 6, 8, 12, len(picks_H)]

for day, TRF in zip(['Day1', 'Day2'], [TRF1, TRF2]):
    for n, contrast in enumerate(contrasts):
        print(contrast)
        if "music" in contrast[0]: 
            color_prod = 'purple'
            color_perc = 'plum'
        elif "speech" in contrast[0]: 
            color_prod = '#008000'
            color_perc = '#90EE90'

        data_production = np.array([TRF[contrast[0]][pick]['score'] for pick in picks_H])
        data_perception = np.array([TRF[contrast[1]][pick]['score'] for pick in picks_H])

        mean_production = np.mean(data_production, axis=1)
        se_production = np.std(data_production, axis=1, ddof=1) / np.sqrt(len(data_production[0]))

        mean_perception = np.mean(data_perception, axis=1)
        se_perception = np.std(data_perception, axis=1, ddof=1) / np.sqrt(len(data_perception[0]))

        # t-test
        t, p = stats.stats.ttest_rel(data_production.T, data_perception.T)
        p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)    
        print(p_corrected < 0.05)
        significance_threshold = 0.05
        significant_channels = np.where(p_corrected < significance_threshold)[0]

        # plot
        fig, ax = plt.subplots(figsize=(12, 8))
        ax2 = ax.twinx()  
    
        ## plot means, se  
        for c in range(len(channel_assignment)-1):
            ax.errorbar(mean_production[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_production[channel_assignment[c]:channel_assignment[c+1]], marker='o', color=color_prod, capsize=8)
            ax.errorbar(mean_perception[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_perception[channel_assignment[c]:channel_assignment[c+1]], marker='o', color=color_perc, capsize=8)
        

        ## plot significant differences
        for s in range(len(picks_H)):
            if p_corrected[s] < 0.05:
                x_pos = - 0.025
                y_pos = s - 0.15
                ax.text(x_pos, y_pos, "*", fontsize=20, color='black', ha='center', va='center', zorder=3)

        ax.set_xlabel('\n r$^2$-coefficients', fontsize=16)
        ax.tick_params(axis='x', labelsize=16)
        ax.set_ylabel('Channels', fontsize=16)
        ax.set_xlim([-0.05, 0.42])
        ax.set_yticks(np.arange(len(picks_H)))
        ax.set_yticklabels(picks_H, fontsize=14)
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        for channel_pos_index in [[0,5], [5,7], [7,11]]: 
            ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1)
        ax2.set_ylim(ax.get_ylim()) 
        channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
        ax2.set_yticks(channel_midpoints)  
        ax2.set_yticklabels(brain_label, size=14, va='center')  

        legend_handles = [
            Line2D([0], [0], marker='o', color='w', markerfacecolor=color_prod, markersize=15, label='Production'),
            Line2D([0], [0], marker='o', color='w', markerfacecolor=color_perc, markersize=15, label='Perception')
        ]

        ax.legend(handles=legend_handles, fontsize=13) 
    plt.show()

### Mixed linear effects model 


#### Model for music

In [ ]:
data = []
for day, TRF in zip(['Day1', 'Day2'], [TRF1, TRF2]):
    for condition in ['produce_music', 'perceive_music_produced']:
        for pick in picks_H:
            score = TRF[condition][pick]['score']
            for sc in score:  
                data.append({'Subject': 'BouNi', 'Condition': condition, 'Day': day, 'Channel': pick, 'Score': sc})

data_music = pd.DataFrame(data)

data_music['Condition'] = pd.Categorical(data_music['Condition'], 
                                          categories=['produce_music', 'perceive_music_produced'],
                                          ordered=True)

In [ ]:
model = smf.mixedlm("Score ~ C(Condition) + C(Day) + C(Condition) * C(Day)", data_music, groups=data_music["Subject"])
result = model.fit()
print("music")
print(result.summary())

#### Model for speech

In [ ]:
data = []
for day, TRF in zip(['Day1', 'Day2'], [TRF1, TRF2]):
    for condition in ['produce_speech', 'perceive_speech_produced']:
        for pick in picks_H:
            score = TRF[condition][pick]['score']
            for sc in score:  
                data.append({'Subject': 'BouNi', 'Condition': condition, 'Day': day, 'Channel': pick, 'Score': sc})

data_speech = pd.DataFrame(data)

data_speech['Condition'] = pd.Categorical(data_speech['Condition'], 
                                          categories=['produce_speech', 'perceive_speech_produced'],
                                          ordered=True)

In [ ]:
model = smf.mixedlm("Score ~ C(Condition) + C(Day) + C(Condition) * C(Day)", data_speech, groups=data_speech["Subject"])
result = model.fit()
print("speech")
print(result.summary().as_text())

## Speech and Music Repetition: First Hearing - Second Hearing 

In [ ]:
# load TRF for control conditions
TRF_control = {}
for condition in condition_list_control: 
        TRF_control[condition] = {}
        print(condition)
        for pick in picks_H: 
                TRF_control[condition][pick] = {}
                TRF_control[condition][pick]['score'] = TRF1[condition][pick]['score']
                TRF_control[condition][pick]['coefs'] = TRF1[condition][pick]['coefs']
                TRF_control[condition][pick]['predicted_Y'] = TRF1[condition][pick]['predicted_Y']
                TRF_control[condition][pick]['times'] = TRF1[condition][pick]['times'] 

In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]
channel_assignment = [0, 6, 8, 12, len(picks_H)]

for n, contrast in enumerate(contrasts_control):
    print(contrast)
    if "music" in contrast[0]: 
        color_prod = 'purple'
        color_perc = 'plum'
    elif "speech" in contrast[0]: 
        color_prod = '#008000'
        color_perc = '#90EE90'

    data_firsthearing = np.array([TRF_control[contrast[0]][pick]['score'] for pick in picks_H])
    data_secondhearing = np.array([TRF_control[contrast[1]][pick]['score'] for pick in picks_H])

    mean_firsthearing = np.mean(data_firsthearing, axis=1)
    se_firsthearing = np.std(data_firsthearing, axis=1, ddof=1) / np.sqrt(len(data_firsthearing[0]))

    mean_secondhearing = np.mean(data_secondhearing, axis=1)
    se_secondhearing = np.std(data_secondhearing, axis=1, ddof=1) / np.sqrt(len(data_secondhearing[0]))

    # t-test
    t, p = stats.ttest_rel(data_firsthearing.T, data_secondhearing.T)
    p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)    
    print(p_corrected < 0.05)
    significance_threshold = 0.05
    significant_channels = np.where(p_corrected < significance_threshold)[0]

    # plot
    fig, ax = plt.subplots(figsize=(12, 8))
    ax2 = ax.twinx()  

    ## plot data points 
    jitter_width = 0.15 
    for i in range(len(data_firsthearing)):
        jitter_prod = np.random.uniform(-jitter_width, jitter_width, size=len(data_firsthearing[i]))
        jitter_perc = np.random.uniform(-jitter_width, jitter_width, size=len(data_secondhearing[i]))

        ax.scatter(data_firsthearing[i], np.full(len(data_firsthearing[i]), i) + jitter_prod,
                alpha=0.7, color=color_prod, s=10)
        ax.scatter(data_secondhearing[i], np.full(len(data_secondhearing[i]), i) + jitter_perc,
                alpha=0.7, color=color_perc, s=10)
        
    ## plot means, se  
    for c in range(len(channel_assignment)-1):
        ax.errorbar(mean_firsthearing[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_firsthearing[channel_assignment[c]:channel_assignment[c+1]], marker='o', linestyle='-', color=color_prod, capsize=8)
        ax.errorbar(mean_secondhearing[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_secondhearing[channel_assignment[c]:channel_assignment[c+1]], marker='s', linestyle='-', color=color_perc, capsize=8)
    
    ## plot significant differences
    for s in range(len(picks_H)):
        if p_corrected[s] < 0.05:
            x_pos = -0.225
            y_pos = s - 0.15
            ax.text(x_pos, y_pos, "*", fontsize=20, color='black', ha='center', va='center', zorder=3)
        

    #ax.set_title(f'r$^2$ Coeffients for {contrast_names_control[n]} \n', fontsize=16)
    ax.set_xlabel('r$^2$ Coefficients', fontsize=14)
    ax.set_ylabel('Channels', fontsize=14)
    ax.set_xlim([-0.25, 0.55])
    ax.set_yticks(np.arange(len(picks_H)))
    ax.set_yticklabels(picks_H, fontsize=10)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    for channel_pos_index in [[0,5], [5,7], [7,11]]: 
        ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1)
    ax2.set_ylim(ax.get_ylim()) 
    channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
    ax2.set_yticks(channel_midpoints)  
    ax2.set_yticklabels(brain_label, size=10, va='center')  

    ax.legend(["First Hearing", "Second Hearing"])
    plt.show()


In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]
channel_assignment = [0, 6, 8, 12, len(picks_H)]

for n, contrast in enumerate(contrasts_control):
    print(contrast)
    if "music" in contrast[0]: 
        color_prod = 'purple'
        color_perc = 'plum'
    elif "speech" in contrast[0]: 
        color_prod = '#008000'
        color_perc = '#90EE90'

    data_firsthearing = np.array([TRF_control[contrast[0]][pick]['score'] for pick in picks_H])
    data_secondhearing = np.array([TRF_control[contrast[1]][pick]['score'] for pick in picks_H])

    mean_firsthearing = np.mean(data_firsthearing, axis=1)
    se_firsthearing = np.std(data_firsthearing, axis=1, ddof=1) / np.sqrt(len(data_firsthearing[0]))

    mean_secondhearing = np.mean(data_secondhearing, axis=1)
    se_secondhearing = np.std(data_secondhearing, axis=1, ddof=1) / np.sqrt(len(data_secondhearing[0]))

    # t-test
    t, p = stats.stats.ttest_rel(data_firsthearing.T, data_secondhearing.T)
    p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)    
    print(p_corrected < 0.05)
    significance_threshold = 0.05
    significant_channels = np.where(p_corrected < significance_threshold)[0]

    # plot
    fig, ax = plt.subplots(figsize=(12, 8))
    ax2 = ax.twinx()  
  
    ## plot means, se  
    for c in range(len(channel_assignment)-1):
        ax.errorbar(mean_firsthearing[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_firsthearing[channel_assignment[c]:channel_assignment[c+1]], marker='o', color=color_prod, capsize=8)
        ax.errorbar(mean_secondhearing[channel_assignment[c]:channel_assignment[c+1]], picks_H[channel_assignment[c]:channel_assignment[c+1]], xerr=se_secondhearing[channel_assignment[c]:channel_assignment[c+1]], marker='o', color=color_perc, capsize=8)
    

    ## plot significant differences
    for s in range(len(picks_H)):
        if p_corrected[s] < 0.05:
            x_pos = - 0.025
            y_pos = s - 0.15
            ax.text(x_pos, y_pos, "*", fontsize=20, color='black', ha='center', va='center', zorder=3)

    ax.set_xlabel('\n r$^2$-coefficients', fontsize=16)
    ax.tick_params(axis='x', labelsize=16)
    ax.set_ylabel('Channels', fontsize=16)
    ax.set_xlim([-0.05, 0.42])
    ax.set_yticks(np.arange(len(picks_H)))
    ax.set_yticklabels(picks_H, fontsize=14)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    for channel_pos_index in [[0,5], [5,7], [7,11]]: 
        ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1)
    ax2.set_ylim(ax.get_ylim()) 
    channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
    ax2.set_yticks(channel_midpoints)  
    ax2.set_yticklabels(brain_label, size=14, va='center')  

    legend_handles = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor=color_prod, markersize=15, label='First Hearing'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor=color_perc, markersize=15, label='Second Hearing')
    ]

    ax.legend(handles=legend_handles, fontsize=13) 
plt.show()

### Linear Mixed Effect Model

#### Model for music

In [ ]:
data = []
for condition in ['perceive_music_new', 'perceive_music_newrepetition']:
    for pick in picks_H:
        score = TRF_control[condition][pick]['score']
        for sc in score:  
            data.append({'Subject': 'BouNi', 'Condition': condition, 'Channel': pick, 'Score': sc})

data_music = pd.DataFrame(data)

data_music['Condition'] = pd.Categorical(data_music['Condition'], 
                                          categories=['perceive_music_new', 'perceive_music_newrepetition'],
                                          ordered=True)         

In [ ]:
model = smf.mixedlm("Score ~ Condition", data_music, groups=data_music["Subject"])
result = model.fit()
print("music")
print(result.summary())

#### Model for speech 

In [ ]:
data = []
for condition in ['perceive_speech_new', 'perceive_speech_newrepetition']:
    for pick in picks_H:
        score = TRF_control[condition][pick]['score']
        for sc in score:  
            data.append({'Subject': 'BouNi', 'Condition': condition, 'Channel': pick, 'Score': sc})

data_speech = pd.DataFrame(data)

data_speech['Condition'] = pd.Categorical(data_speech['Condition'], 
                                          categories=['perceive_speech_new', 'perceive_speech_newrepetition'],
                                          ordered=True)

In [ ]:
model = smf.mixedlm("Score ~ Condition", data_speech, groups=data_speech["Subject"])
result = model.fit()
print("speech")
print(result.summary())